# Image Generation with GANs - Complete Tutorial

**Author:** Anik Tahabilder  
**Project:** 16 of 22 - Kaggle ML Portfolio  
**Difficulty:** 9/10 | **Learning Value:** 9/10

---

## What Will You Learn?

This tutorial builds **Generative Adversarial Networks** from scratch, progressing from simple to advanced:

| Part | Model | Output Quality |
|------|-------|----------------|
| 1 | Basic GAN | Blurry digits |
| 2 | DCGAN | Clear digits |
| 3 | Conditional GAN | Specific digits |
| 4 | DCGAN on Fashion | Clothing items |

---

## GAN Architecture Overview

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         GAN ARCHITECTURE                                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│   ┌─────────────┐      ┌─────────────┐      ┌─────────────┐            │
│   │   RANDOM    │      │             │      │  GENERATED  │            │
│   │   NOISE     │ ───► │  GENERATOR  │ ───► │   IMAGE     │            │
│   │    (z)      │      │    G(z)     │      │   G(z)      │            │
│   └─────────────┘      └─────────────┘      └──────┬──────┘            │
│                                                     │                   │
│                                                     ▼                   │
│                                            ┌───────────────┐            │
│   ┌─────────────┐                          │               │            │
│   │    REAL     │                          │ DISCRIMINATOR │            │
│   │   IMAGE     │ ────────────────────────►│               │            │
│   │    (x)      │                          │   D(x)        │            │
│   └─────────────┘                          └───────┬───────┘            │
│                                                     │                   │
│                                                     ▼                   │
│                                            ┌───────────────┐            │
│                                            │  Real or Fake │            │
│                                            │   (0 or 1)    │            │
│                                            └───────────────┘            │
│                                                                         │
│   The GENERATOR tries to fool the DISCRIMINATOR                        │
│   The DISCRIMINATOR tries to catch fakes                               │
│   They compete and both get better!                                    │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

## GAN Training Process

```
┌─────┐   ┌─────┐   ┌─────┐   ┌─────┐   ┌─────┐   ┌─────┐
│  1  │──►│  2  │──►│  3  │──►│  4  │──►│  5  │──►│  6  │
└──┬──┘   └──┬──┘   └──┬──┘   └──┬──┘   └──┬──┘   └──┬──┘
   │         │         │         │         │         │
   ▼         ▼         ▼         ▼         ▼         ▼
Define    Select     Train D   Generate  Train D   Train G
Problem   Arch.      on Real   Fakes     on Fake   with D
```

---

## Table of Contents

1. [Part 1: GAN Fundamentals](#part1)
2. [Part 2: Basic GAN on MNIST](#part2)
3. [Part 3: Deep Convolutional GAN (DCGAN)](#part3)
4. [Part 4: Conditional GAN (cGAN)](#part4)
5. [Part 5: GAN on Fashion-MNIST](#part5)
6. [Part 6: Training Tips & Tricks](#part6)
7. [Part 7: Evaluation & Results](#part7)
8. [Part 8: Summary](#part8)

---

<a id='part1'></a>
# Part 1: GAN Fundamentals

---

## 1.1 What is a GAN?

**Generative Adversarial Network** = Two neural networks competing:

| Network | Role | Analogy |
|---------|------|--------|
| **Generator (G)** | Creates fake images | Counterfeiter making fake money |
| **Discriminator (D)** | Detects fakes | Police detecting counterfeit |

## 1.2 The Adversarial Game

```
Generator's Goal:    Maximize D(G(z)) → Make D think fakes are real
Discriminator's Goal: Maximize D(x), Minimize D(G(z)) → Correctly classify
```

## 1.3 GAN Loss Function

**Minimax Game:**
```
min_G max_D V(D, G) = E[log D(x)] + E[log(1 - D(G(z)))]

Where:
- D(x) = Discriminator's output for real image x
- G(z) = Generator's output for noise z
- D(G(z)) = Discriminator's output for fake image
```

## 1.4 Training Dynamics

| Stage | Generator | Discriminator |
|-------|-----------|---------------|
| Early | Random noise | Easily spots fakes |
| Middle | Blurry images | Gets harder to tell |
| Late | Realistic images | 50/50 guess (equilibrium) |

In [ ]:
# ============================================================
# SETUP AND IMPORTS
# ============================================================
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("="*70)
print("IMAGE GENERATION WITH GANs - TUTORIAL")
print("="*70)
print(f"PyTorch: {torch.__version__}")
print(f"Device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("\nAll libraries loaded!")

In [ ]:
# ============================================================
# LOAD MNIST DATASET
# ============================================================
print("="*70)
print("LOADING MNIST DATASET")
print("="*70)

# Hyperparameters
BATCH_SIZE = 128
IMAGE_SIZE = 28
CHANNELS = 1
LATENT_DIM = 100  # Size of noise vector

# Transform: normalize to [-1, 1] for tanh activation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # (x - 0.5) / 0.5 → [-1, 1]
])

# Load MNIST
mnist_train = datasets.MNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform
)

dataloader = DataLoader(
    mnist_train, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=0
)

print(f"Dataset size: {len(mnist_train):,}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Number of batches: {len(dataloader)}")
print(f"Image size: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"Latent dimension: {LATENT_DIM}")

# Show sample images
sample_batch, sample_labels = next(iter(dataloader))
print(f"\nSample batch shape: {sample_batch.shape}")

# Visualize
fig, axes = plt.subplots(2, 8, figsize=(12, 3))
for i, ax in enumerate(axes.flat):
    img = sample_batch[i].squeeze().numpy()
    img = (img + 1) / 2  # Denormalize to [0, 1]
    ax.imshow(img, cmap='gray')
    ax.axis('off')
    ax.set_title(f'{sample_labels[i].item()}')
plt.suptitle('Sample MNIST Images', fontweight='bold')
plt.tight_layout()
plt.show()

---

<a id='part2'></a>
# Part 2: Basic GAN on MNIST

---

## 2.1 Simple GAN Architecture

```
GENERATOR:
┌────────┐   ┌─────────┐   ┌─────────┐   ┌─────────┐   ┌─────────┐
│ Noise  │──►│ Linear  │──►│ Linear  │──►│ Linear  │──►│  Tanh   │
│  100   │   │   256   │   │   512   │   │   784   │   │ [-1,1]  │
└────────┘   └─────────┘   └─────────┘   └─────────┘   └─────────┘

DISCRIMINATOR:
┌────────┐   ┌─────────┐   ┌─────────┐   ┌─────────┐   ┌─────────┐
│ Image  │──►│ Linear  │──►│ Linear  │──►│ Linear  │──►│ Sigmoid │
│  784   │   │   512   │   │   256   │   │    1    │   │  [0,1]  │
└────────┘   └─────────┘   └─────────┘   └─────────┘   └─────────┘
```

In [ ]:
# ============================================================
# BASIC GAN - GENERATOR
# ============================================================
print("="*70)
print("BASIC GAN ARCHITECTURE")
print("="*70)

class BasicGenerator(nn.Module):
    """
    Basic Generator using fully connected layers.
    
    Input: Random noise vector (batch, latent_dim)
    Output: Generated image (batch, channels, height, width)
    """
    
    def __init__(self, latent_dim=100, img_shape=(1, 28, 28)):
        super().__init__()
        
        self.img_shape = img_shape
        img_size = int(np.prod(img_shape))  # 1 * 28 * 28 = 784
        
        self.model = nn.Sequential(
            # Layer 1: 100 → 256
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            
            # Layer 2: 256 → 512
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            
            # Layer 3: 512 → 1024
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            
            # Output: 1024 → 784
            nn.Linear(1024, img_size),
            nn.Tanh()  # Output in [-1, 1]
        )
    
    def forward(self, z):
        """Generate images from noise."""
        img_flat = self.model(z)
        img = img_flat.view(img_flat.size(0), *self.img_shape)
        return img


class BasicDiscriminator(nn.Module):
    """
    Basic Discriminator using fully connected layers.
    
    Input: Image (batch, channels, height, width)
    Output: Probability of being real (batch, 1)
    """
    
    def __init__(self, img_shape=(1, 28, 28)):
        super().__init__()
        
        img_size = int(np.prod(img_shape))  # 784
        
        self.model = nn.Sequential(
            # Layer 1: 784 → 512
            nn.Linear(img_size, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            # Layer 2: 512 → 256
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            # Output: 256 → 1
            nn.Linear(256, 1),
            nn.Sigmoid()  # Output probability [0, 1]
        )
    
    def forward(self, img):
        """Classify image as real or fake."""
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# Create models
img_shape = (CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
generator = BasicGenerator(LATENT_DIM, img_shape).to(device)
discriminator = BasicDiscriminator(img_shape).to(device)

print("\nGENERATOR:")
print(generator)

print("\nDISCRIMINATOR:")
print(discriminator)

# Count parameters
g_params = sum(p.numel() for p in generator.parameters())
d_params = sum(p.numel() for p in discriminator.parameters())
print(f"\nGenerator parameters: {g_params:,}")
print(f"Discriminator parameters: {d_params:,}")

In [ ]:
# Test the models
print("="*70)
print("TESTING MODELS")
print("="*70)

# Generate random noise
test_noise = torch.randn(4, LATENT_DIM, device=device)
print(f"Noise shape: {test_noise.shape}")

# Generate fake images
with torch.no_grad():
    fake_images = generator(test_noise)
print(f"Generated image shape: {fake_images.shape}")

# Discriminator output
with torch.no_grad():
    d_output = discriminator(fake_images)
print(f"Discriminator output shape: {d_output.shape}")
print(f"Discriminator predictions: {d_output.squeeze().cpu().numpy()}")

# Visualize random generator output (before training)
fig, axes = plt.subplots(1, 4, figsize=(8, 2))
for i, ax in enumerate(axes):
    img = fake_images[i].squeeze().cpu().numpy()
    img = (img + 1) / 2  # Denormalize
    ax.imshow(img, cmap='gray')
    ax.axis('off')
plt.suptitle('Generator Output (Before Training) - Random Noise', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# TRAINING BASIC GAN
# ============================================================
print("="*70)
print("TRAINING BASIC GAN")
print("="*70)

def train_basic_gan(generator, discriminator, dataloader, epochs=20, lr=0.0002):
    """
    Train Basic GAN.
    
    Training Steps:
    1. Train Discriminator on real images (label = 1)
    2. Train Discriminator on fake images (label = 0)
    3. Train Generator (fool discriminator, label = 1)
    """
    
    # Loss function
    criterion = nn.BCELoss()
    
    # Optimizers
    optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    # Training history
    g_losses = []
    d_losses = []
    
    # Fixed noise for visualization
    fixed_noise = torch.randn(64, LATENT_DIM, device=device)
    
    print(f"\nTraining for {epochs} epochs...")
    print(f"{'Epoch':>6} {'D Loss':>10} {'G Loss':>10} {'D(x)':>8} {'D(G(z))':>10}")
    print("-" * 50)
    
    for epoch in range(epochs):
        epoch_g_loss = 0
        epoch_d_loss = 0
        d_x_total = 0
        d_gz_total = 0
        
        for i, (real_imgs, _) in enumerate(dataloader):
            batch_size = real_imgs.size(0)
            real_imgs = real_imgs.to(device)
            
            # Labels
            real_labels = torch.ones(batch_size, 1, device=device)
            fake_labels = torch.zeros(batch_size, 1, device=device)
            
            # ---------------------
            # Train Discriminator
            # ---------------------
            optimizer_D.zero_grad()
            
            # Loss on real images
            d_real = discriminator(real_imgs)
            d_real_loss = criterion(d_real, real_labels)
            
            # Generate fake images
            z = torch.randn(batch_size, LATENT_DIM, device=device)
            fake_imgs = generator(z)
            
            # Loss on fake images
            d_fake = discriminator(fake_imgs.detach())
            d_fake_loss = criterion(d_fake, fake_labels)
            
            # Total discriminator loss
            d_loss = (d_real_loss + d_fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()
            
            # ---------------------
            # Train Generator
            # ---------------------
            optimizer_G.zero_grad()
            
            # Generate fake images
            z = torch.randn(batch_size, LATENT_DIM, device=device)
            fake_imgs = generator(z)
            
            # Generator wants discriminator to think fakes are real
            g_output = discriminator(fake_imgs)
            g_loss = criterion(g_output, real_labels)
            
            g_loss.backward()
            optimizer_G.step()
            
            # Track metrics
            epoch_d_loss += d_loss.item()
            epoch_g_loss += g_loss.item()
            d_x_total += d_real.mean().item()
            d_gz_total += d_fake.mean().item()
        
        # Average losses
        n_batches = len(dataloader)
        avg_d_loss = epoch_d_loss / n_batches
        avg_g_loss = epoch_g_loss / n_batches
        avg_d_x = d_x_total / n_batches
        avg_d_gz = d_gz_total / n_batches
        
        g_losses.append(avg_g_loss)
        d_losses.append(avg_d_loss)
        
        print(f"{epoch+1:>6} {avg_d_loss:>10.4f} {avg_g_loss:>10.4f} {avg_d_x:>8.4f} {avg_d_gz:>10.4f}")
    
    return g_losses, d_losses, fixed_noise

# Train the GAN
g_losses, d_losses, fixed_noise = train_basic_gan(
    generator, discriminator, dataloader, epochs=20
)

print("\nTraining complete!")

In [ ]:
# Visualize training progress
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss curves
ax1 = axes[0]
ax1.plot(g_losses, label='Generator Loss', color='blue')
ax1.plot(d_losses, label='Discriminator Loss', color='red')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Generate final images
ax2 = axes[1]
generator.eval()
with torch.no_grad():
    fake_imgs = generator(fixed_noise[:16]).cpu()
    
grid = make_grid(fake_imgs, nrow=4, normalize=True, padding=2)
ax2.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax2.axis('off')
ax2.set_title('Generated Images (Basic GAN)', fontweight='bold')

plt.tight_layout()
plt.show()

---

<a id='part3'></a>
# Part 3: Deep Convolutional GAN (DCGAN)

---

## 3.1 Why DCGAN?

| Basic GAN | DCGAN |
|-----------|-------|
| Fully connected layers | Convolutional layers |
| Loses spatial information | Preserves spatial structure |
| Blurry outputs | Sharper images |
| Harder to train | More stable training |

## 3.2 DCGAN Architecture Rules

1. Replace pooling with strided convolutions (D) and transposed convolutions (G)
2. Use BatchNorm in both G and D
3. Remove fully connected hidden layers
4. Use ReLU in G (except output: Tanh)
5. Use LeakyReLU in D

```
DCGAN GENERATOR:
┌────────┐   ┌───────────┐   ┌───────────┐   ┌───────────┐   ┌────────┐
│ Noise  │──►│ ConvT 4x4 │──►│ ConvT 4x4 │──►│ ConvT 4x4 │──►│ Image  │
│  100   │   │  256 ch   │   │  128 ch   │   │   64 ch   │   │ 28x28  │
└────────┘   └───────────┘   └───────────┘   └───────────┘   └────────┘
               Upsample       Upsample        Upsample
```

In [ ]:
# ============================================================
# DCGAN ARCHITECTURE
# ============================================================
print("="*70)
print("DCGAN ARCHITECTURE")
print("="*70)

class DCGenerator(nn.Module):
    """
    DCGAN Generator using transposed convolutions.
    
    Progressively upsamples from noise to image.
    """
    
    def __init__(self, latent_dim=100, channels=1, feature_maps=64):
        super().__init__()
        
        self.init_size = 7  # Initial size before upsampling
        self.l1 = nn.Sequential(
            nn.Linear(latent_dim, feature_maps * 4 * self.init_size * self.init_size)
        )
        
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(feature_maps * 4),
            
            # Upsample: 7x7 → 14x14
            nn.ConvTranspose2d(feature_maps * 4, feature_maps * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_maps * 2),
            nn.ReLU(True),
            
            # Upsample: 14x14 → 28x28
            nn.ConvTranspose2d(feature_maps * 2, feature_maps, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_maps),
            nn.ReLU(True),
            
            # Output: 28x28 → 28x28
            nn.Conv2d(feature_maps, channels, 3, 1, 1),
            nn.Tanh()
        )
    
    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.size(0), -1, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class DCDiscriminator(nn.Module):
    """
    DCGAN Discriminator using strided convolutions.
    
    Progressively downsamples image to single probability.
    """
    
    def __init__(self, channels=1, feature_maps=64):
        super().__init__()
        
        self.model = nn.Sequential(
            # Input: 1x28x28
            # Downsample: 28x28 → 14x14
            nn.Conv2d(channels, feature_maps, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Downsample: 14x14 → 7x7
            nn.Conv2d(feature_maps, feature_maps * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_maps * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Downsample: 7x7 → 3x3
            nn.Conv2d(feature_maps * 2, feature_maps * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_maps * 4),
            nn.LeakyReLU(0.2, inplace=True),
        )
        
        # Calculate output size
        self.adv_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(feature_maps * 4 * 3 * 3, 1),
            nn.Sigmoid()
        )
    
    def forward(self, img):
        features = self.model(img)
        validity = self.adv_layer(features)
        return validity

# Create DCGAN models
dc_generator = DCGenerator(LATENT_DIM, CHANNELS).to(device)
dc_discriminator = DCDiscriminator(CHANNELS).to(device)

# Weight initialization (important for DCGANs)
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

dc_generator.apply(weights_init)
dc_discriminator.apply(weights_init)

print("\nDCGAN GENERATOR:")
print(dc_generator)

print("\nDCGAN DISCRIMINATOR:")
print(dc_discriminator)

# Count parameters
g_params = sum(p.numel() for p in dc_generator.parameters())
d_params = sum(p.numel() for p in dc_discriminator.parameters())
print(f"\nDCGAN Generator parameters: {g_params:,}")
print(f"DCGAN Discriminator parameters: {d_params:,}")

In [ ]:
# ============================================================
# TRAIN DCGAN
# ============================================================
print("="*70)
print("TRAINING DCGAN")
print("="*70)

def train_dcgan(generator, discriminator, dataloader, epochs=30, lr=0.0002):
    """
    Train DCGAN with improved training strategy.
    """
    
    criterion = nn.BCELoss()
    
    optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    g_losses = []
    d_losses = []
    
    fixed_noise = torch.randn(64, LATENT_DIM, device=device)
    
    # Store generated images at different epochs
    image_history = []
    
    print(f"\nTraining DCGAN for {epochs} epochs...")
    print(f"{'Epoch':>6} {'D Loss':>10} {'G Loss':>10} {'D(x)':>8} {'D(G(z))':>10}")
    print("-" * 50)
    
    for epoch in range(epochs):
        epoch_g_loss = 0
        epoch_d_loss = 0
        d_x_total = 0
        d_gz_total = 0
        
        for i, (real_imgs, _) in enumerate(dataloader):
            batch_size = real_imgs.size(0)
            real_imgs = real_imgs.to(device)
            
            # Soft labels for more stable training
            real_labels = torch.ones(batch_size, 1, device=device) * 0.9
            fake_labels = torch.zeros(batch_size, 1, device=device) + 0.1
            
            # ---------------------
            # Train Discriminator
            # ---------------------
            optimizer_D.zero_grad()
            
            d_real = discriminator(real_imgs)
            d_real_loss = criterion(d_real, real_labels)
            
            z = torch.randn(batch_size, LATENT_DIM, device=device)
            fake_imgs = generator(z)
            
            d_fake = discriminator(fake_imgs.detach())
            d_fake_loss = criterion(d_fake, fake_labels)
            
            d_loss = (d_real_loss + d_fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()
            
            # ---------------------
            # Train Generator
            # ---------------------
            optimizer_G.zero_grad()
            
            z = torch.randn(batch_size, LATENT_DIM, device=device)
            fake_imgs = generator(z)
            g_output = discriminator(fake_imgs)
            
            # Generator wants discriminator output to be 1 (real)
            g_loss = criterion(g_output, torch.ones(batch_size, 1, device=device))
            
            g_loss.backward()
            optimizer_G.step()
            
            epoch_d_loss += d_loss.item()
            epoch_g_loss += g_loss.item()
            d_x_total += d_real.mean().item()
            d_gz_total += d_fake.mean().item()
        
        n_batches = len(dataloader)
        avg_d_loss = epoch_d_loss / n_batches
        avg_g_loss = epoch_g_loss / n_batches
        avg_d_x = d_x_total / n_batches
        avg_d_gz = d_gz_total / n_batches
        
        g_losses.append(avg_g_loss)
        d_losses.append(avg_d_loss)
        
        # Save generated images periodically
        if (epoch + 1) % 5 == 0 or epoch == 0:
            generator.eval()
            with torch.no_grad():
                gen_imgs = generator(fixed_noise[:16]).cpu()
            image_history.append((epoch + 1, gen_imgs))
            generator.train()
        
        print(f"{epoch+1:>6} {avg_d_loss:>10.4f} {avg_g_loss:>10.4f} {avg_d_x:>8.4f} {avg_d_gz:>10.4f}")
    
    return g_losses, d_losses, fixed_noise, image_history

# Train DCGAN
dc_g_losses, dc_d_losses, dc_fixed_noise, image_history = train_dcgan(
    dc_generator, dc_discriminator, dataloader, epochs=30
)

print("\nTraining complete!")

In [ ]:
# Visualize training progression
print("="*70)
print("DCGAN TRAINING PROGRESSION")
print("="*70)

n_images = len(image_history)
fig, axes = plt.subplots(1, n_images, figsize=(4 * n_images, 4))

for i, (epoch, imgs) in enumerate(image_history):
    grid = make_grid(imgs, nrow=4, normalize=True, padding=2)
    axes[i].imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(f'Epoch {epoch}', fontweight='bold')

plt.suptitle('DCGAN Training Progression', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Compare Basic GAN vs DCGAN
print("="*70)
print("BASIC GAN vs DCGAN COMPARISON")
print("="*70)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Basic GAN losses
ax1 = axes[0, 0]
ax1.plot(g_losses, label='Generator', color='blue')
ax1.plot(d_losses, label='Discriminator', color='red')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Basic GAN Training Loss', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# DCGAN losses
ax2 = axes[0, 1]
ax2.plot(dc_g_losses, label='Generator', color='blue')
ax2.plot(dc_d_losses, label='Discriminator', color='red')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('DCGAN Training Loss', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Basic GAN outputs
ax3 = axes[1, 0]
generator.eval()
with torch.no_grad():
    basic_imgs = generator(fixed_noise[:16]).cpu()
grid = make_grid(basic_imgs, nrow=4, normalize=True, padding=2)
ax3.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax3.axis('off')
ax3.set_title('Basic GAN Generated Images', fontweight='bold')

# DCGAN outputs
ax4 = axes[1, 1]
dc_generator.eval()
with torch.no_grad():
    dc_imgs = dc_generator(dc_fixed_noise[:16]).cpu()
grid = make_grid(dc_imgs, nrow=4, normalize=True, padding=2)
ax4.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax4.axis('off')
ax4.set_title('DCGAN Generated Images', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nObservation: DCGAN produces sharper, more recognizable digits!")

---

<a id='part4'></a>
# Part 4: Conditional GAN (cGAN)

---

## 4.1 What is Conditional GAN?

**Regular GAN:** Generates random images (no control over what digit)

**Conditional GAN:** Generates specific images based on a condition (label)

```
Regular GAN:  noise → Generator → random digit
Conditional GAN: noise + label → Generator → specific digit
```

## 4.2 cGAN Architecture

| Component | Input | Output |
|-----------|-------|--------|
| Generator | noise + label | image of that class |
| Discriminator | image + label | real/fake for that class |

In [ ]:
# ============================================================
# CONDITIONAL GAN (cGAN)
# ============================================================
print("="*70)
print("CONDITIONAL GAN (cGAN)")
print("="*70)

N_CLASSES = 10  # MNIST has 10 digits

class ConditionalGenerator(nn.Module):
    """
    Conditional Generator - generates images conditioned on class label.
    """
    
    def __init__(self, latent_dim=100, n_classes=10, img_shape=(1, 28, 28)):
        super().__init__()
        
        self.img_shape = img_shape
        
        # Label embedding
        self.label_emb = nn.Embedding(n_classes, n_classes)
        
        self.model = nn.Sequential(
            nn.Linear(latent_dim + n_classes, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )
    
    def forward(self, noise, labels):
        # Concatenate noise and label embedding
        label_input = self.label_emb(labels)
        gen_input = torch.cat((noise, label_input), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *self.img_shape)
        return img


class ConditionalDiscriminator(nn.Module):
    """
    Conditional Discriminator - classifies images conditioned on class label.
    """
    
    def __init__(self, n_classes=10, img_shape=(1, 28, 28)):
        super().__init__()
        
        self.label_emb = nn.Embedding(n_classes, n_classes)
        
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)) + n_classes, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, img, labels):
        img_flat = img.view(img.size(0), -1)
        label_input = self.label_emb(labels)
        d_input = torch.cat((img_flat, label_input), -1)
        validity = self.model(d_input)
        return validity

# Create cGAN models
cgan_generator = ConditionalGenerator(LATENT_DIM, N_CLASSES, img_shape).to(device)
cgan_discriminator = ConditionalDiscriminator(N_CLASSES, img_shape).to(device)

print("Conditional GAN created!")
print(f"Generator can generate specific digits (0-9)")
print(f"Discriminator verifies if image matches the label")

In [ ]:
# ============================================================
# TRAIN CONDITIONAL GAN
# ============================================================
print("="*70)
print("TRAINING CONDITIONAL GAN")
print("="*70)

def train_cgan(generator, discriminator, dataloader, epochs=30, lr=0.0002):
    """
    Train Conditional GAN.
    """
    criterion = nn.BCELoss()
    
    optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    g_losses = []
    d_losses = []
    
    # Fixed noise and labels for visualization
    fixed_noise = torch.randn(100, LATENT_DIM, device=device)
    fixed_labels = torch.arange(10, device=device).repeat(10)  # 0,1,2,...,9 repeated 10 times
    
    print(f"\nTraining cGAN for {epochs} epochs...")
    print(f"{'Epoch':>6} {'D Loss':>10} {'G Loss':>10}")
    print("-" * 30)
    
    for epoch in range(epochs):
        epoch_g_loss = 0
        epoch_d_loss = 0
        
        for i, (real_imgs, labels) in enumerate(dataloader):
            batch_size = real_imgs.size(0)
            real_imgs = real_imgs.to(device)
            labels = labels.to(device)
            
            real_labels = torch.ones(batch_size, 1, device=device)
            fake_labels = torch.zeros(batch_size, 1, device=device)
            
            # ---------------------
            # Train Discriminator
            # ---------------------
            optimizer_D.zero_grad()
            
            d_real = discriminator(real_imgs, labels)
            d_real_loss = criterion(d_real, real_labels)
            
            z = torch.randn(batch_size, LATENT_DIM, device=device)
            gen_labels = torch.randint(0, N_CLASSES, (batch_size,), device=device)
            fake_imgs = generator(z, gen_labels)
            
            d_fake = discriminator(fake_imgs.detach(), gen_labels)
            d_fake_loss = criterion(d_fake, fake_labels)
            
            d_loss = (d_real_loss + d_fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()
            
            # ---------------------
            # Train Generator
            # ---------------------
            optimizer_G.zero_grad()
            
            z = torch.randn(batch_size, LATENT_DIM, device=device)
            gen_labels = torch.randint(0, N_CLASSES, (batch_size,), device=device)
            fake_imgs = generator(z, gen_labels)
            
            g_output = discriminator(fake_imgs, gen_labels)
            g_loss = criterion(g_output, real_labels)
            
            g_loss.backward()
            optimizer_G.step()
            
            epoch_d_loss += d_loss.item()
            epoch_g_loss += g_loss.item()
        
        n_batches = len(dataloader)
        g_losses.append(epoch_g_loss / n_batches)
        d_losses.append(epoch_d_loss / n_batches)
        
        if (epoch + 1) % 5 == 0:
            print(f"{epoch+1:>6} {d_losses[-1]:>10.4f} {g_losses[-1]:>10.4f}")
    
    return g_losses, d_losses, fixed_noise, fixed_labels

# Train cGAN
cgan_g_losses, cgan_d_losses, cgan_fixed_noise, cgan_fixed_labels = train_cgan(
    cgan_generator, cgan_discriminator, dataloader, epochs=30
)

print("\nTraining complete!")

In [ ]:
# Generate specific digits with cGAN
print("="*70)
print("GENERATING SPECIFIC DIGITS WITH cGAN")
print("="*70)

cgan_generator.eval()

fig, axes = plt.subplots(10, 10, figsize=(12, 12))

with torch.no_grad():
    for digit in range(10):
        # Generate 10 samples of each digit
        z = torch.randn(10, LATENT_DIM, device=device)
        labels = torch.full((10,), digit, dtype=torch.long, device=device)
        generated = cgan_generator(z, labels).cpu()
        
        for j in range(10):
            img = generated[j].squeeze().numpy()
            img = (img + 1) / 2
            axes[digit, j].imshow(img, cmap='gray')
            axes[digit, j].axis('off')
        
        # Label each row
        axes[digit, 0].set_ylabel(f'{digit}', fontsize=12, fontweight='bold', rotation=0, labelpad=15)

plt.suptitle('Conditional GAN: Generating Specific Digits (0-9)', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

print("\nEach row shows 10 different samples of the same digit!")
print("The cGAN learned to generate specific digits on demand.")

---

<a id='part5'></a>
# Part 5: GAN on Fashion-MNIST

---

Let's train DCGAN on a more challenging dataset - Fashion-MNIST (clothing items).

In [ ]:
# ============================================================
# LOAD FASHION-MNIST
# ============================================================
print("="*70)
print("DCGAN ON FASHION-MNIST")
print("="*70)

# Fashion-MNIST classes
fashion_classes = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Load Fashion-MNIST
fashion_train = datasets.FashionMNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform
)

fashion_dataloader = DataLoader(
    fashion_train, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=0
)

print(f"Dataset size: {len(fashion_train):,}")
print(f"Classes: {fashion_classes}")

# Show samples
sample_batch, sample_labels = next(iter(fashion_dataloader))

fig, axes = plt.subplots(2, 8, figsize=(12, 3))
for i, ax in enumerate(axes.flat):
    img = sample_batch[i].squeeze().numpy()
    img = (img + 1) / 2
    ax.imshow(img, cmap='gray')
    ax.axis('off')
    ax.set_title(f'{fashion_classes[sample_labels[i]]}', fontsize=8)
plt.suptitle('Sample Fashion-MNIST Images', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Create new DCGAN for Fashion-MNIST
fashion_generator = DCGenerator(LATENT_DIM, CHANNELS).to(device)
fashion_discriminator = DCDiscriminator(CHANNELS).to(device)

fashion_generator.apply(weights_init)
fashion_discriminator.apply(weights_init)

print("Training DCGAN on Fashion-MNIST...")
print("(This may take a few minutes)\n")

# Train on Fashion-MNIST
fashion_g_losses, fashion_d_losses, fashion_fixed_noise, fashion_history = train_dcgan(
    fashion_generator, fashion_discriminator, fashion_dataloader, epochs=30
)

print("\nTraining complete!")

In [ ]:
# Visualize Fashion-MNIST generation
print("="*70)
print("FASHION-MNIST GENERATED IMAGES")
print("="*70)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Training loss
ax1 = axes[0, 0]
ax1.plot(fashion_g_losses, label='Generator', color='blue')
ax1.plot(fashion_d_losses, label='Discriminator', color='red')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Fashion-MNIST DCGAN Training', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Real images
ax2 = axes[0, 1]
real_batch, _ = next(iter(fashion_dataloader))
grid = make_grid(real_batch[:16], nrow=4, normalize=True, padding=2)
ax2.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax2.axis('off')
ax2.set_title('Real Fashion Images', fontweight='bold')

# Generated images
ax3 = axes[1, 0]
fashion_generator.eval()
with torch.no_grad():
    gen_fashion = fashion_generator(fashion_fixed_noise[:16]).cpu()
grid = make_grid(gen_fashion, nrow=4, normalize=True, padding=2)
ax3.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax3.axis('off')
ax3.set_title('Generated Fashion Images', fontweight='bold')

# Training progression
ax4 = axes[1, 1]
if fashion_history:
    final_epoch, final_imgs = fashion_history[-1]
    grid = make_grid(final_imgs, nrow=4, normalize=True, padding=2)
    ax4.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax4.axis('off')
ax4.set_title('Final Generated Images', fontweight='bold')

plt.tight_layout()
plt.show()

---

<a id='part6'></a>
# Part 6: Training Tips & Tricks

---

In [ ]:
# ============================================================
# GAN TRAINING TIPS
# ============================================================
print("="*70)
print("GAN TRAINING TIPS & TRICKS")
print("="*70)

print("""
COMMON PROBLEMS AND SOLUTIONS:
==============================

1. MODE COLLAPSE:
   Problem: Generator produces same image repeatedly
   ┌─────────────────────────────────────────────────────────────┐
   │ Solutions:                                                   │
   │ - Use minibatch discrimination                              │
   │ - Add noise to discriminator inputs                         │
   │ - Use different learning rates for G and D                  │
   │ - Use Wasserstein loss (WGAN)                               │
   └─────────────────────────────────────────────────────────────┘

2. TRAINING INSTABILITY:
   Problem: Loss oscillates wildly, no convergence
   ┌─────────────────────────────────────────────────────────────┐
   │ Solutions:                                                   │
   │ - Use soft labels (0.9 instead of 1.0)                      │
   │ - Use label smoothing                                        │
   │ - Gradient clipping                                          │
   │ - Use spectral normalization                                 │
   │ - Lower learning rate                                        │
   └─────────────────────────────────────────────────────────────┘

3. DISCRIMINATOR TOO STRONG:
   Problem: D loss → 0, G can't learn
   ┌─────────────────────────────────────────────────────────────┐
   │ Solutions:                                                   │
   │ - Train G more times per D update                           │
   │ - Add dropout to D                                          │
   │ - Reduce D capacity                                         │
   │ - Use instance noise                                        │
   └─────────────────────────────────────────────────────────────┘

4. VANISHING GRADIENTS:
   Problem: G receives near-zero gradients
   ┌─────────────────────────────────────────────────────────────┐
   │ Solutions:                                                   │
   │ - Use LeakyReLU instead of ReLU                             │
   │ - Use non-saturating loss: -log(D(G(z)))                    │
   │ - Use Wasserstein loss                                      │
   └─────────────────────────────────────────────────────────────┘

BEST PRACTICES:
===============
✓ Normalize images to [-1, 1], use Tanh in G output
✓ Use BatchNorm in both G and D (except D input, G output)
✓ Use LeakyReLU (slope 0.2) in discriminator
✓ Use Adam optimizer with β1=0.5
✓ Use strided convolutions instead of pooling
✓ Initialize weights from N(0, 0.02)
✓ Track both D(x) and D(G(z)) during training
✓ Save generated samples periodically
""")

---

<a id='part7'></a>
# Part 7: Evaluation & Results

---

In [ ]:
# ============================================================
# FINAL RESULTS COMPARISON
# ============================================================
print("="*70)
print("FINAL RESULTS - ALL MODELS")
print("="*70)

fig = plt.figure(figsize=(16, 12))

# Create grid spec
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Real MNIST
ax1 = fig.add_subplot(gs[0, 0])
real_batch, _ = next(iter(dataloader))
grid = make_grid(real_batch[:16], nrow=4, normalize=True, padding=2)
ax1.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax1.axis('off')
ax1.set_title('Real MNIST', fontweight='bold')

# 2. Basic GAN
ax2 = fig.add_subplot(gs[0, 1])
generator.eval()
with torch.no_grad():
    noise = torch.randn(16, LATENT_DIM, device=device)
    basic_imgs = generator(noise).cpu()
grid = make_grid(basic_imgs, nrow=4, normalize=True, padding=2)
ax2.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax2.axis('off')
ax2.set_title('Basic GAN', fontweight='bold')

# 3. DCGAN
ax3 = fig.add_subplot(gs[0, 2])
dc_generator.eval()
with torch.no_grad():
    dc_imgs = dc_generator(noise).cpu()
grid = make_grid(dc_imgs, nrow=4, normalize=True, padding=2)
ax3.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax3.axis('off')
ax3.set_title('DCGAN', fontweight='bold')

# 4. Conditional GAN - specific digits
ax4 = fig.add_subplot(gs[1, 0])
cgan_generator.eval()
with torch.no_grad():
    # Generate one of each digit
    z = torch.randn(10, LATENT_DIM, device=device)
    labels = torch.arange(10, device=device)
    cgan_imgs = cgan_generator(z, labels).cpu()
grid = make_grid(cgan_imgs, nrow=5, normalize=True, padding=2)
ax4.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax4.axis('off')
ax4.set_title('cGAN (digits 0-9)', fontweight='bold')

# 5. Real Fashion-MNIST
ax5 = fig.add_subplot(gs[1, 1])
real_fashion, _ = next(iter(fashion_dataloader))
grid = make_grid(real_fashion[:16], nrow=4, normalize=True, padding=2)
ax5.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax5.axis('off')
ax5.set_title('Real Fashion-MNIST', fontweight='bold')

# 6. Generated Fashion
ax6 = fig.add_subplot(gs[1, 2])
fashion_generator.eval()
with torch.no_grad():
    fashion_imgs = fashion_generator(noise).cpu()
grid = make_grid(fashion_imgs, nrow=4, normalize=True, padding=2)
ax6.imshow(grid.permute(1, 2, 0).numpy(), cmap='gray')
ax6.axis('off')
ax6.set_title('DCGAN Fashion', fontweight='bold')

# 7. Training comparison
ax7 = fig.add_subplot(gs[2, :])
epochs = range(1, len(dc_g_losses) + 1)
ax7.plot(epochs, dc_g_losses, 'b-', label='DCGAN MNIST - G', alpha=0.7)
ax7.plot(epochs, dc_d_losses, 'b--', label='DCGAN MNIST - D', alpha=0.7)
ax7.plot(epochs, fashion_g_losses, 'r-', label='DCGAN Fashion - G', alpha=0.7)
ax7.plot(epochs, fashion_d_losses, 'r--', label='DCGAN Fashion - D', alpha=0.7)
ax7.set_xlabel('Epoch')
ax7.set_ylabel('Loss')
ax7.set_title('Training Loss Comparison', fontweight='bold')
ax7.legend(loc='upper right')
ax7.grid(True, alpha=0.3)

plt.suptitle('GAN Image Generation Results', fontweight='bold', fontsize=16)
plt.tight_layout()
plt.show()

---

<a id='part8'></a>
# Part 8: Summary

---

In [ ]:
# Final summary
print("="*70)
print("IMAGE GENERATION WITH GANs - SUMMARY")
print("="*70)

print("""
WHAT WE LEARNED:
================

1. GAN FUNDAMENTALS:
   ┌─────────────────────────────────────────────────────────────┐
   │ Generator (G): Creates fake images from noise               │
   │ Discriminator (D): Classifies real vs fake                  │
   │ Training: Adversarial game - G vs D                         │
   │ Goal: G fools D, both improve together                      │
   └─────────────────────────────────────────────────────────────┘

2. GAN ARCHITECTURES:
   ┌─────────────────┬─────────────────────────────────────────┐
   │ Basic GAN       │ Fully connected layers, simple          │
   │ DCGAN           │ Convolutional layers, sharper images    │
   │ Conditional GAN │ Control what to generate                │
   └─────────────────┴─────────────────────────────────────────┘

3. TRAINING PROCESS:
   Step 1: Train D on real images (label = 1)
   Step 2: Train D on fake images (label = 0)
   Step 3: Train G to fool D (wants D output = 1)
   Repeat!

4. KEY LOSS FUNCTIONS:
   D_loss = -[log(D(x)) + log(1 - D(G(z)))]
   G_loss = -log(D(G(z)))  # or log(1 - D(G(z)))

5. DCGAN RULES:
   ✓ Use strided convolutions (no pooling)
   ✓ Use BatchNorm (except D input, G output)
   ✓ Use LeakyReLU in D, ReLU in G
   ✓ Use Tanh output in G (images in [-1, 1])

6. EVALUATION METRICS:
   - Inception Score (IS): Higher is better
   - Fréchet Inception Distance (FID): Lower is better
   - Visual inspection: Most practical!

MODERN GAN VARIANTS:
====================
- WGAN: Wasserstein loss for stable training
- StyleGAN: High-quality face generation
- CycleGAN: Unpaired image-to-image translation
- BigGAN: Large-scale image synthesis
- Diffusion Models: Now state-of-the-art (DALL-E, Stable Diffusion)
""")

print("\nMODEL COMPARISON:")
print("-" * 60)
print(f"{'Model':<20} {'Dataset':<15} {'Quality':<15}")
print("-" * 60)
print(f"{'Basic GAN':<20} {'MNIST':<15} {'Blurry digits':<15}")
print(f"{'DCGAN':<20} {'MNIST':<15} {'Clear digits':<15}")
print(f"{'Conditional GAN':<20} {'MNIST':<15} {'Specific digits':<15}")
print(f"{'DCGAN':<20} {'Fashion-MNIST':<15} {'Clothing items':<15}")

print("\n" + "="*70)

## Algorithm & Method Taxonomy

### GAN Architectures

| Architecture | Key Feature | Best For |
|--------------|-------------|----------|
| **Basic GAN** | Fully connected | Simple distributions |
| **DCGAN** | Convolutional | Image generation |
| **cGAN** | Conditional input | Class-specific generation |
| **WGAN** | Wasserstein loss | Stable training |
| **StyleGAN** | Style mixing | High-quality faces |
| **CycleGAN** | Cycle consistency | Image translation |

### Loss Functions

| Loss | Formula | Pros/Cons |
|------|---------|----------|
| **BCE** | -log(D(x)) | Standard, can saturate |
| **Non-saturating** | -log(D(G(z))) | Better gradients for G |
| **Wasserstein** | E[D(x)] - E[D(G(z))] | Stable, needs clipping |
| **Hinge** | max(0, 1-D(x)) | Works well in practice |

### Training Stability Techniques

| Technique | Purpose |
|-----------|--------|
| **Label smoothing** | Prevent D overconfidence |
| **Instance noise** | Regularize D |
| **Spectral norm** | Stabilize D gradients |
| **Two timescale** | Different LR for G and D |
| **Progressive growing** | Start small, grow resolution |

---

## Checklist

- [x] Understand GAN fundamentals (G vs D game)
- [x] Implement basic GAN from scratch
- [x] Implement DCGAN with convolutions
- [x] Implement conditional GAN
- [x] Train on MNIST and Fashion-MNIST
- [x] Know common training problems and solutions
- [x] Generate recognizable images

---

**End of Image Generation with GANs Tutorial**